# Description

This notebook contains examples of CCXT functionality.

## Imports

In [1]:
%load_ext autoreload
%autoreload 2
import logging
import pprint

import ccxt

import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import helpers.hsecrets as hsecret

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-ecb44a46-f8b8-46ce-a90a-f443f9a2ed5f.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.3' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='gp_scratch'
  hash='588f2cb41'
  # Last commits:
    *   588f2cb41 saggese  Merge                                                             ( 4 minutes ago) Fri Jun 30 09:33:36 2023  (HEAD -> gp_scratch, origin/gp_scratch)
    |\  
    * | f9f65aa6a saggese  Checkpoint                                                        ( 5 minutes ago) Fri Jun 30 09:32:44 2023           
    | * 6a2db717d GP Saggese Checkpoint (#4685) 

# CCXT

## Resources
- https://github.com/ccxt/ccxt
- https://docs.ccxt.com/#/README
- https://github.com/ccxt/ccxt#readme
- https://ccxt.readthedocs.io/en/latest/index.html
- https://ccxt.readthedocs.io/en/latest/manual.html

## Intro

- CCXT = CryptoCurrency eXchange Trading library

- Connect with cryptocurrency exchanges and trade
- Connect with payment processing services
- Access to market data
- Algorithmic trading
- Strategy backtesting
- Bot programming

- Normalized API for cross-exchange analytics and arbitrage

- There is an async mode using asyncio
  ```
  import ccxt.async_support as ccxt
  ```

- Proxy: in case Cloudflare or your country / IP is rejected
  - Of course an intermediary adds latency
  - Server that acts as an intermediary between the client requesting a resource and the server

## Usage

// https://github.com/ccxt/ccxt#usage

- Public API
  - Unrestricted access to public information for exchanges without account or
    API key
  - E.g.,
    - Market data
    - Order books
    - Price feeds
    - Trade history

- Private API
  - Obtain an API key from exchange website by signing up
    - You might need personal info and verification
  - Manage personal account info
  - Query account balance
  - Query orders
  - Trade
  - Deposit and withdraw fiat and crypto funds

- CCXT supports REST APIs for all exchanges
- CCXT Pro supports WebSocket and FIX

# Exchanges

From https://docs.ccxt.com/#/README?id=exchanges

- Each class implements the public and private API for a particular crypto
  exchange
- The `Exchange` class shares a set of common methods

- Some exchanges offer:
  - Margin trading (i.e., leverage)
  - Derivatives (e.g., futures and options)
  - Dark pools, OTC trading
  - Merchant APIs

- Testnets and mainnets envs
    - Some exchanges offer a separated API for:
      - Testing purposes (aka "sandboxes", "staging environments", "testnets")
      - Trading with real assets (aka "mainnets", "production environments")
    - Typically the sandbox has the same API as the production API but with a
      different URL (and maybe different market)

- Exchange structure
    - Every exchange has properties that can be overridden in the constructor

    - `id`: default id for identification purposes (typically a lower case string)
    - `name`: human-readable exchange name
    - `countries`: where the exchange is operating from
    - `urls`: URLs for private and public APIs, main website, documentation
    - `version`: version of the current exchange API
    - `api`: API endpoints exposed by a crypto exchange
    - `has`: array of exchange capabilities
    - `timeframes`: frequency of available bars (e.g., minutes)
    - `rateLimit`: minimum delay between two consecutive requests
    - `markets`: dictionary of markets indexed by symbols
    - `symbols`: list of symbols available with an exchange
    - `currencies`: array of currencies available

- Exchange metadata
    - Each exchange has a `has` with flags about the exchange capabilities

- Rate limit
    - Exchanges track your user / IP address to throttle querying the API too
      frequently
    - You need to stay under the rate limit to avoid being banned
      - Typical limits are 1 or 2 requests per second
    - CCXT has an experimental rate-limiter that throttles in background
      ```
      exchange.enableRateLimit = True
      ```
      - The state is inside the class instance so one should have a single class

In [12]:
# Print all exchanges.
print(len(ccxt.exchanges), ccxt.exchanges)

117 ['aax', 'alpaca', 'ascendex', 'bequant', 'bibox', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bit2c', 'bitbank', 'bitbay', 'bitbns', 'bitcoincom', 'bitfinex', 'bitfinex2', 'bitflyer', 'bitforex', 'bitget', 'bithumb', 'bitmart', 'bitmex', 'bitopro', 'bitpanda', 'bitrue', 'bitso', 'bitstamp', 'bitstamp1', 'bittrex', 'bitvavo', 'bkex', 'bl3p', 'blockchaincom', 'btcalpha', 'btcbox', 'btcex', 'btcmarkets', 'btctradeua', 'btcturk', 'buda', 'bw', 'bybit', 'bytetrade', 'cex', 'coinbase', 'coinbaseprime', 'coinbasepro', 'coincheck', 'coinex', 'coinfalcon', 'coinmate', 'coinone', 'coinspot', 'crex24', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'exmo', 'flowbtc', 'fmfwio', 'gate', 'gateio', 'gemini', 'hitbtc', 'hitbtc3', 'hollaex', 'huobi', 'huobijp', 'huobipro', 'idex', 'independentreserve', 'indodax', 'itbit', 'kraken', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'lbank2', 'liquid', 'luno', 'lykke', 'mercado', 'mexc', 'mexc3', 'ndax', 'nova

In [84]:
# Create Binance exchange.
exchange_id = "binance"
mode = "test"
contract_type = "futures"

# Select credentials for provided exchange.
if mode == "test":
    secrets_id = exchange_id + "_sandbox"
else:
    secrets_id = exchange_id
exchange_params = hsecret.get_secret(secrets_id)

# Enable rate limit.
exchange_params["rateLimit"] = True

# Log into futures/spot market.
if contract_type == "futures":
    exchange_params["options"] = {"defaultType": "future"}

# Create a CCXT Exchange class object.
ccxt_exchange = getattr(ccxt, exchange_id)
print(hprint.to_str("ccxt_exchange"))
exchange = ccxt_exchange(exchange_params)
print(hprint.to_str("exchange"))
if mode == "test":
    exchange.set_sandbox_mode(True)
    _LOG.warning("Running in sandbox mode")
hdbg.dassert(
    exchange.checkRequiredCredentials(),
    msg="Required credentials not passed",
)

ccxt_exchange=<class 'ccxt.binance.binance'>
exchange=ccxt.binance()
WARNING Running in sandbox mode


## Exchange properties

In [23]:
print("exchange=", exchange, type(exchange))

exchange= Binance <class 'ccxt.binance.binance'>


In [88]:
# # Print some properties of the exchange.
# var_names = ["exchange.id", 
#              "exchange.name",
#              "exchange.countries",
#              #"exchange.urls",
#              "exchange.version",
#              "exchange.timeframes",
#              "exchange.timeout",
#              "exchange.rateLimit",
#              "exchange.symbols",
#              "exchange.currencies"]
# for var_name in var_names:
#     print(hprint.to_str(var_name, mode="pprint_color"))

In [89]:
# Name in user-land to identify the exchange.
hprint.pprint_color(exchange.id)

'binance'


In [90]:
# Human readable name.
hprint.pprint_color(exchange.name)

'Binance'


In [91]:
# Which countries the exchange is operating from.
hprint.pprint_color(exchange.countries)

['JP', 'MT']


In [92]:
# Version identifier for exchange API.
hprint.pprint_color(exchange.version)

None


In [93]:
# timeframes for fetchOHLCV().
# TODO(gp): It seems that it has 1s resolution.
hprint.pprint_color(exchange.timeframes)

{'12h': '12h',
 '15m': '15m',
 '1M': '1M',
 '1d': '1d',
 '1h': '1h',
 '1m': '1m',
 '1s': '1s',
 '1w': '1w',
 '2h': '2h',
 '30m': '30m',
 '3d': '3d',
 '3m': '3m',
 '4h': '4h',
 '5m': '5m',
 '6h': '6h',
 '8h': '8h'}


In [95]:
hprint.pprint_color(exchange.requiredCredentialsCredentials)

{'apiKey': True,
 'login': False,
 'password': False,
 'privateKey': False,
 'secret': True,
 'token': False,
 'twofa': False,
 'uid': False,
 'walletAddress': False}


In [96]:
# Exchange decimal precision.
hprint.pprint_color(exchange.precisionMode)

2


In [87]:
hprint.pprint_color(exchange.urls)

{'api': {'dapiPrivate': 'https://testnet.binancefuture.com/dapi/v1',
         'dapiPublic': 'https://testnet.binancefuture.com/dapi/v1',
         'eapiPrivate': 'https://testnet.binanceops.com/eapi/v1',
         'eapiPublic': 'https://testnet.binanceops.com/eapi/v1',
         'fapiPrivate': 'https://testnet.binancefuture.com/fapi/v1',
         'fapiPrivateV2': 'https://testnet.binancefuture.com/fapi/v2',
         'fapiPublic': 'https://testnet.binancefuture.com/fapi/v1',
         'private': 'https://testnet.binance.vision/api/v3',
         'public': 'https://testnet.binance.vision/api/v3',
         'v1': 'https://testnet.binance.vision/api/v1'},
 'apiBackup': {'dapiData': 'https://dapi.binance.com/futures/data',
               'dapiPrivate': 'https://dapi.binance.com/dapi/v1',
               'dapiPrivateV2': 'https://dapi.binance.com/dapi/v2',
               'dapiPublic': 'https://dapi.binance.com/dapi/v1',
               'eapiPrivate': 'https://eapi.binance.com/eapi/v1',
             

In [86]:
hprint.pprint_color(exchange.api)

{'dapiData': {'get': {'basis': 1,
                      'globalLongShortAccountRatio': 1,
                      'openInterestHist': 1,
                      'takerBuySellVol': 1,
                      'topLongShortAccountRatio': 1,
                      'topLongShortPositionRatio': 1}},
 'dapiPrivate': {'delete': {'allOpenOrders': 1,
                            'batchOrders': 5,
                            'listenKey': 1,
                            'order': 1},
                 'get': {'account': 5,
                         'adlQuantile': 5,
                         'allOrders': {'cost': 20, 'noSymbol': 40},
                         'balance': 1,
                         'forceOrders': {'cost': 20, 'noSymbol': 50},
                         'income': 20,
                         'leverageBracket': 1,
                         'openOrder': 1,
                         'openOrders': {'cost': 1, 'noSymbol': 5},
                         'order': 1,
                         'orderAmendment': 

In [39]:
exchange.loadMarkets()
print(hprint.list_to_str(exchange.markets.keys(), tag="market_list"))

market_list: (226) BTC/USDT ETH/USDT BCH/USDT XRP/USDT EOS/USDT LTC/USDT TRX/USDT ETC/USDT LINK/USDT XLM/USDT ADA/USDT DASH/USDT ZEC/USDT XTZ/USDT BNB/USDT ATOM/USDT ONT/USDT IOTA/USDT BAT/USDT VET/USDT NEOUSDT QTUM/USDT IOST/USDT THETA/USDT ALGO/USDT ZIL/USDT KNC/USDT ZRX/USDT COMP/USDT OMG/USDT DOGE/USDT SXP/USDT KAVA/USDT BAND/USDT RLC/USDT WAVES/USDT MKR/USDT SNX/USDT DOT/USDT DEFI/USDT YFI/USDT BAL/USDT CRV/USDT TRBUSDT RUNE/USDT SUSHI/USDT SRMUSDT EGLD/USDT SOL/USDT ICXUSDT STORJ/USDT UNI/USDT AVAX/USDT FTM/USDT HNT/USDT ENJ/USDT FLM/USDT TOMO/USDT REN/USDT KSM/USDT NEAR/USDT AAVE/USDT FIL/USDT RSRUSDT LRC/USDT MATIC/USDT OCEAN/USDT BELUSDT CTKUSDT AXS/USDT ALPHAUSDT ZEN/USDT SKL/USDT GRT/USDT BNTUSDT 1INCH/USDT UNFI/USDT BTC/BUSD CHZ/USDT SAND/USDT ANKR/USDT LIT/USDT REEF/USDT RVNUSDT SFPUSDT XEMUSDT BTCSTUSDT COTI/USDT CHR/USDT MANA/USDT ALICE/USDT HBARUSDT ONE/USDT LINA/USDT STMX/USDT DENT/USDT CELR/USDT HOT/USDT MTL/USDT OGN/USDT NKNUSDT DGBUSDT 1000SHIB/USDT BAKEUSDT GTCUSDT

In [58]:
print(hprint.list_to_str(exchange.currencies, tag="currencies"))

currencies: (196) 1000FLOKI 1000LUNC 1000PEPE 1000SHIB 1000XEC 1INCH AAVE ACH ADA AGIX ALGO ALICE ALPHA AMB ANC ANKR ANT APE API3 APT AR ARB ARPA ASTR ATA ATOM AUCTION AUDIO AVAX AXS BAKE BAL BAND BAT BCH BEL BLUEBIRD BLUR BNB BNT BNX BTC BTCDOM BTCST BUSD C98 CELO CELR CFX CHR CHZ CKB COMBO COMP COTI CRV CTK CTSI CVX DAR DASH DEFI DENT DGB DOGE DOT DUSK DYDX EDU EGLD ENJ ENS EOS ETC ETH FET FIL FLM FLOW FOOTBALL FTM FXS GAL GALA GMT GMX GRT GTC HBAR HFT HIGH HNT HOOK HOT ICP ICP2 ICX ID IDEX IMX INJ IOST IOTA IOTX JASMY JOE KAVA KEY KLAY KNC KSM LDO LEVER LINA LINK LIT LPT LQTY LRC LTC LUNA2 MAGIC MANA MASK MATIC MAV MDT MINA MKR MTL NEAR NEO NKN NMR OCEAN OGN OMG ONE ONT OP PEOPLE PERP PHB QNT QTUM RAD RDNT REEF REN RLC RNDR ROSE RSR RUNE RVN SAND SFP SKL SNX SOL SPELL SRM SSV STG STMX STORJ STX SUI SUSHI SXP T THETA TOMO TRB TRU TRX UMA UNFI UNI USDC USDT VET WAVE WAVES WOO XEM XLM XMR XRP XTZ XVS YFI ZEC ZEN ZIL ZRX



In [56]:
hprint.pprint_color(exchange.commonCurrencies)

{'BCC': 'BCC', 'BCHABC': 'BCH', 'BCHSV': 'BSV', 'XBT': 'BTC', 'YOYO': 'YOYOW'}



## Exchange metadata

In [44]:
# Flags for exchange capabilities (true, false, emulated).
# CORS = cross-origin resource sharing.
hprint.pprint_color(exchange.has)

{'CORS': None,
 'addMargin': True,
 'borrowMargin': True,
 'cancelAllOrders': True,
 'cancelOrder': True,
 'cancelOrders': None,
 'createDepositAddress': False,
 'createLimitOrder': True,
 'createMarketOrder': True,
 'createOrder': True,
 'createPostOnlyOrder': None,
 'createReduceOnlyOrder': True,
 'createStopLimitOrder': True,
 'createStopMarketOrder': False,
 'createStopOrder': True,
 'editOrder': 'emulated',
 'fetchAccounts': None,
 'fetchBalance': True,
 'fetchBidsAsks': True,
 'fetchBorrowInterest': True,
 'fetchBorrowRate': True,
 'fetchBorrowRateHistories': False,
 'fetchBorrowRateHistory': True,
 'fetchBorrowRates': False,
 'fetchBorrowRatesPerSymbol': False,
 'fetchCanceledOrders': False,
 'fetchClosedOrder': False,
 'fetchClosedOrders': 'emulated',
 'fetchCurrencies': True,
 'fetchDeposit': False,
 'fetchDepositAddress': True,
 'fetchDepositAddresses': False,
 'fetchDepositAddressesByNetwork': False,
 'fetchDeposits': True,
 'fetchFundingFee': None,
 'fetchFundingFees': None

## Rate limit

# Markets

- Valuables
    - Valuables are exchanged at each market
      - E.g.,
        - instruments
        - symbols
        - assets
        - trading pairs
        - currencies
        - tokens
        - contract

- Exchange and Market
    - Every Exchange offers multiple Markets
    - A Market is usually a pair of currencies (e.g., crypto, fiat)

## Currency structure

- Each currency has an associated dictionary
    - `id`: currency id within the exchange
    - `code`: `cctx` representation of the currency
    - `name`: human readable currency name
    - `fee`: withdrawal fee
    - `active`: indicates whether trading and funding this currency is possible
    - `info`: dictionary of non-common market properties
    - `precision`
    - `limits`: min and max for withdrawals

## Market structure

- `id` string representing the instrument within the exchange (e.g., `btcusd`)
- `baseId` (e.g., `btc`), `quoteId` (e.g., `usd`) are exchange-specific ids
- `symbol` string code representing the trading pair
  - E.g., typically `BaseCurrency/QuoteCurrency` (e.g., `BTC/USD`)
  - This is standard in `ccxt`
- `base` (e.g., `BTC`) / `quote` (`USD`) standardized currency code
- `active`: indicates whether trading this market is possible
  - The cache of the markets should be refreshed periodically
- `maker`: maker fees paid when you provide liquidity to the exchange
  - E.g., you make an order and someone else fills it
  - A negative fee means a rebate
- `taker`: taker fees paid when you take liquidity from the exchange (i.e., you
  fill someone else's order)
- `tierBased`: whether the fee depends on your trading tier (e.g., amount of
  trading done over a period of time)
- `info`: non-common market properties
- `precision`: precision used for price, amount, and cost
  - E.g., decimal places, significant digits, tick size

In [121]:
# A market is an associative array.

#market_id = exchange.markets_by_id["1000FLOKIUSDT"]
market_id = exchange.markets_by_id["ETHUSDT"]
print(type(market_id))
hprint.pprint_color(market_id)

<class 'dict'>
{'active': True,
 'base': 'ETH',
 'baseId': 'ETH',
 'contract': True,
 'contractSize': 1.0,
 'delivery': False,
 'expiry': None,
 'expiryDatetime': None,
 'future': True,
 'id': 'ETHUSDT',
 'info': {'baseAsset': 'ETH',
          'baseAssetPrecision': '8',
          'contractType': 'PERPETUAL',
          'deliveryDate': '4133404800000',
          'filters': [{'filterType': 'PRICE_FILTER',
                       'maxPrice': '85514.26',
                       'minPrice': '16.76',
                       'tickSize': '0.01'},
                      {'filterType': 'LOT_SIZE',
                       'maxQty': '10000',
                       'minQty': '0.001',
                       'stepSize': '0.001'},
                      {'filterType': 'MARKET_LOT_SIZE',
                       'maxQty': '10000',
                       'minQty': '0.001',
                       'stepSize': '0.001'},
                      {'filterType': 'MAX_NUM_ORDERS', 'limit': '200'},
                      {'

In [123]:
# Trade instrument within the exchange. This is the internal representation of each exchange.
print(market["id"])

ETHUSDT


In [124]:
# Trade instrument in CCXT user-land (unified). Typically referrend as "base/quote".
print(market["symbol"])

ETH/USDT


In [125]:
# Market ids (unified)
print(market["base"], market["quote"])

ETH USDT


In [126]:
# Symbol ids (not unified).
print(market["baseId"], market["quoteId"])

ETH USDT


In [132]:
print(hprint.to_str('market["active"] market["maker"] market["taker"] market["percentage"]'))

market["active"]=True, market["maker"]=0.0002, market["taker"]=0.0004, market["percentage"]=True


In [133]:
# Market-specific properties.
hprint.pprint_color(market["info"])

{'baseAsset': 'ETH',
 'baseAssetPrecision': '8',
 'contractType': 'PERPETUAL',
 'deliveryDate': '4133404800000',
 'filters': [{'filterType': 'PRICE_FILTER',
              'maxPrice': '85514.26',
              'minPrice': '16.76',
              'tickSize': '0.01'},
             {'filterType': 'LOT_SIZE',
              'maxQty': '10000',
              'minQty': '0.001',
              'stepSize': '0.001'},
             {'filterType': 'MARKET_LOT_SIZE',
              'maxQty': '10000',
              'minQty': '0.001',
              'stepSize': '0.001'},
             {'filterType': 'MAX_NUM_ORDERS', 'limit': '200'},
             {'filterType': 'MAX_NUM_ALGO_ORDERS', 'limit': '10'},
             {'filterType': 'MIN_NOTIONAL', 'notional': '5.0'},
             {'filterType': 'PERCENT_PRICE',
              'multiplierDecimal': '4',
              'multiplierDown': '0.9000',
              'multiplierUp': '1.1000'}],
 'liquidationFee': '0.020000',
 'maintMarginPercent': '2.5000',
 'marginAsset': '

## Network structure

## Precision and limits

In [143]:
# limits = min, max for prices / amounts (aka volumes) / costs (= price * amount)
# precision = precision for prices / amounts / costs accepted in order values when placing orders
# They are not related.

hprint.pprint_color(market["limits"], tag="limits", sep="\n")
hprint.pprint_color(market["precision"], tag="precision", sep="\n")

limits= 
{'amount': {'max': 10000.0, 'min': 0.001},
 'cost': {'max': None, 'min': 5.0},
 'leverage': {'max': None, 'min': None},
 'market': {'max': 10000.0, 'min': 0.001},
 'price': {'max': 85514.26, 'min': 16.76}}
precision= 
{'amount': 3, 'base': 8, 'price': 2, 'quote': 8}


In [137]:
# Min / max amount (i.e., volume) for an order.
print(market["limits"]["amount"])

{'min': 0.001, 'max': 10000.0}


In [135]:
# How many decimal digits.

{'amount': 3, 'base': 8, 'price': 2, 'quote': 8}


In [ ]:
# Each exchange has their own way of rounding and truncating.

In [144]:
exchange.precisionMode

2

In [147]:
ccxt.TICK_SIZE

4

In [146]:
ccxt.SIGNIFICANT_DIGITS

3

In [145]:
ccxt.DECIMAL_PLACES

2

In [149]:
# From https://docs.ccxt.com/#/README?id=formatting-to-precision
#ccxt.base.decimal_to_precision.amount_to_precision(symbol, amount)

AttributeError: 'list' object has no attribute 'decimal_to_precision'

## Loading markets

In [150]:
markets = exchange.load_markets()

## Symbols and Market Ids

### Symbols and market ids

* Currency code
- = a code of 3 to 5 uppercase letters
- E.g., `BTC`, `ETH`, `USD`, `XRP`

* Symbol
- = a pair of currencies separated by a slash
  - E.g., `BTC/USD`
- The first currency is called the "base currency"
- The second currency is called the "quote currency"
  - BASE / QUOTE

* Market Ids
- Market ids are unique per exchange and are used in REST request-response
  - E.g., the same BTC/USD pair can be called in different ways on different
    markets (e.g., `BTCUSD`, `btc/usd`)
- `CCTX` abstracts market ids into standardized symbols

* Market symbol vs market ids
- "Market symbols" are the abstract representation
- "Market ids" are specific of each market

### Methods for markets and currencies

// notebook

### Naming consistency

* Products
- Some exchanges call markets as "pairs" or "products"
- `CCXT` considers each exchange as having one or more "trading markets"
  - Each market has an `id` and a `symbol`
  - Most symbols are typically a currency pair

* Exchange -> Markets -> Symbols -> Currencies
- The logic is:
  - Exchange (name of the exchange, e.g., Binance)
  - Markets (a "product" that is traded, e.g., the pair `BTC/USD`)
  - Symbols (a pair of traded currencies separated by slash)
  - Currencies (the currency code, e.g., `BTC` and `USD`)

- The same currency:
  - can have different names on different exchanges
  - has changed name over time (e.g., `XBT` -> `BTC`, `USD` = `USDT`)

* Expiring / perpetual futures
- Aka "swaps"
- Futures market symbol have:
  - Underlying currency
  - Quoting currency
  - Settlement currency
  - Identifier for a settlement date (typically as YYMMDD)

- E.g., `BTC/USDT:BTC-211225`
  - BTC/USDT futures contract settled in BTC (inverse) on 2021-12-25
- E.g., `BTC/USDT:USDT-211225`
  - BTC/USDT futures contract settled in USDT (linear, vanilla)

* Perpetual futures
- Aka "perpetual swaps"
- E.g., `BTC/USDT:BTC`

In [97]:
exchange.load_markets();

In [59]:
# Get the market structure.
market = exchange.markets["ETH/USDT"]
hprint.pprint_color(market)

{'active': True,
 'base': 'ETH',
 'baseId': 'ETH',
 'contract': True,
 'contractSize': 1.0,
 'delivery': False,
 'expiry': None,
 'expiryDatetime': None,
 'feeSide': 'get',
 'future': True,
 'id': 'ETHUSDT',
 'info': {'baseAsset': 'ETH',
          'baseAssetPrecision': '8',
          'contractType': 'PERPETUAL',
          'deliveryDate': '4133404800000',
          'filters': [{'filterType': 'PRICE_FILTER',
                       'maxPrice': '85514.26',
                       'minPrice': '16.76',
                       'tickSize': '0.01'},
                      {'filterType': 'LOT_SIZE',
                       'maxQty': '10000',
                       'minQty': '0.001',
                       'stepSize': '0.001'},
                      {'filterType': 'MARKET_LOT_SIZE',
                       'maxQty': '10000',
                       'minQty': '0.001',
                       'stepSize': '0.001'},
                      {'filterType': 'MAX_NUM_ORDERS', 'limit': '200'},
                    

In [101]:
# Print a subset of interesting values for Market structure.
var_names = [
    "id",
    "symbol",
    "base",
    "quote",
    "baseId",
    "quoteId",
    "active",
    "maker",
    "taker",
    "tierBased",
    "info",
    "precision",
    "limits",
]
for var_name in var_names:
    print(f"--> {var_name}=", hprint.pprint_pformat(market[var_name]))

--> id= 'ETHUSDT'
--> symbol= 'ETH/USDT'
--> base= 'ETH'
--> quote= 'USDT'
--> baseId= 'ETH'
--> quoteId= 'USDT'
--> active= True
--> maker= 0.0002
--> taker= 0.0004
--> tierBased= False
--> info= {'baseAsset': 'ETH',
 'baseAssetPrecision': '8',
 'contractType': 'PERPETUAL',
 'deliveryDate': '4133404800000',
 'filters': [{'filterType': 'PRICE_FILTER',
              'maxPrice': '85514.26',
              'minPrice': '16.76',
              'tickSize': '0.01'},
             {'filterType': 'LOT_SIZE',
              'maxQty': '10000',
              'minQty': '0.001',
              'stepSize': '0.001'},
             {'filterType': 'MARKET_LOT_SIZE',
              'maxQty': '10000',
              'minQty': '0.001',
              'stepSize': '0.001'},
             {'filterType': 'MAX_NUM_ORDERS', 'limit': '200'},
             {'filterType': 'MAX_NUM_ALGO_ORDERS', 'limit': '10'},
             {'filterType': 'MIN_NOTIONAL', 'notional': '5.0'},
             {'filterType': 'PERCENT_PRICE',
        

In [103]:
# Print all the symbols in one exchange.
symbols = exchange.symbols
print(symbols)

['1000FLOKIUSDT', '1000LUNC/BUSD', '1000LUNC/USDT', '1000PEPE/USDT', '1000SHIB/BUSD', '1000SHIB/USDT', '1000XEC/USDT', '1INCH/USDT', 'AAVE/USDT', 'ACH/USDT', 'ADA/BUSD', 'ADA/USDT', 'AGIX/BUSD', 'AGIX/USDT', 'ALGO/USDT', 'ALICE/USDT', 'ALPHAUSDT', 'AMB', 'AMB/BUSD', 'AMB/USDT', 'ANC/BUSD', 'ANKR/USDT', 'ANT/USDT', 'APE/USDT', 'API3/USDT', 'APT/BUSD', 'APT/USDT', 'AR/USDT', 'ARB/BUSD', 'ARB/USDT', 'ARPA/USDT', 'ASTR/USDT', 'ATA/USDT', 'ATOM/USDT', 'AUCTION/BUSD', 'AUDIO/USDT', 'AVAX/USDT', 'AXS/USDT', 'BAKEUSDT', 'BAL/USDT', 'BAND/USDT', 'BAT/USDT', 'BCH/USDT', 'BELUSDT', 'BLUEBIRD/USDT', 'BLUR/USDT', 'BNB/BUSD', 'BNB/USDT', 'BNTUSDT', 'BNX/USDT', 'BTC/BUSD', 'BTC/USDT', 'BTCDOM/USDT', 'BTCSTUSDT', 'BTCUSDT_230929', 'C98/USDT', 'CELO/USDT', 'CELR/USDT', 'CFX/USDT', 'CHR/USDT', 'CHZ/USDT', 'CKB/USDT', 'COMBO/USDT', 'COMP/USDT', 'COTI/USDT', 'CRV/USDT', 'CTKUSDT', 'CTSIUSDT', 'CVX/USDT', 'DAR/USDT', 'DASH/USDT', 'DEFI/USDT', 'DENT/USDT', 'DGBUSDT', 'DOGE/USDT', 'DOGEBUSD', 'DOT/BUSD', 'DO

In [105]:
# Print a dictionary of all currencies.
currencies = exchange.currencies
hprint.pprint_color(currencies)

{'1000FLOKI': {'code': '1000FLOKI',
               'id': '1000FLOKI',
               'numericId': None,
               'precision': 8},
 '1000LUNC': {'code': '1000LUNC',
              'id': '1000LUNC',
              'numericId': None,
              'precision': 8},
 '1000PEPE': {'code': '1000PEPE',
              'id': '1000PEPE',
              'numericId': None,
              'precision': 8},
 '1000SHIB': {'code': '1000SHIB',
              'id': '1000SHIB',
              'numericId': None,
              'precision': 8},
 '1000XEC': {'code': '1000XEC',
             'id': '1000XEC',
             'numericId': None,
             'precision': 8},
 '1INCH': {'code': '1INCH', 'id': '1INCH', 'numericId': None, 'precision': 8},
 'AAVE': {'code': 'AAVE', 'id': 'AAVE', 'numericId': None, 'precision': 8},
 'ACH': {'code': 'ACH', 'id': 'ACH', 'numericId': None, 'precision': 8},
 'ADA': {'code': 'ADA', 'id': 'ADA', 'numericId': None, 'precision': 8},
 'AGIX': {'code': 'AGIX', 'id': 'AGIX', 'numericI

In [116]:
#market_id = exchange.markets_by_id["1000FLOKIUSDT"]
market_id = exchange.markets_by_id["ETHUSDT"]
print(type(market_id))
hprint.pprint_color(market_id)

<class 'dict'>
{'active': True,
 'base': 'ETH',
 'baseId': 'ETH',
 'contract': True,
 'contractSize': 1.0,
 'delivery': False,
 'expiry': None,
 'expiryDatetime': None,
 'future': True,
 'id': 'ETHUSDT',
 'info': {'baseAsset': 'ETH',
          'baseAssetPrecision': '8',
          'contractType': 'PERPETUAL',
          'deliveryDate': '4133404800000',
          'filters': [{'filterType': 'PRICE_FILTER',
                       'maxPrice': '85514.26',
                       'minPrice': '16.76',
                       'tickSize': '0.01'},
                      {'filterType': 'LOT_SIZE',
                       'maxQty': '10000',
                       'minQty': '0.001',
                       'stepSize': '0.001'},
                      {'filterType': 'MARKET_LOT_SIZE',
                       'maxQty': '10000',
                       'minQty': '0.001',
                       'stepSize': '0.001'},
                      {'filterType': 'MAX_NUM_ORDERS', 'limit': '200'},
                      {'

In [120]:
market_id["symbol"]

'ETH/USDT'

## Market cache force reload

# Implicit API methods

## API methods / endpoints

- API methods / endpoints
    - Each exchange offers a set of API methods (aka "endpoints") that are HTTP URLs
      for querying various types of information
    - All endpoints return JSON responses

- E.g., an endpoint for:
  - getting a list of markets from an exchange
  - retrieving an order book
  - retrieving trade history
  - cancelling orders
  
- Endpoints are defined in `api` property of an exchange

## Implicit API methods

- In practice each API method is mapped on callable Python function
- Each function can be called with a dictionary of parameters and return an
  unparsed JSON from the exchange API
- The method is available in both camelCase and under_score notation

## Public / private / unified

- Each `Exchange` implements:
  - a public / private API for all endpoints
  - a unified API supporting a subset of common methods

- One should:
  - use unified methods
  - use the private method as fallback

## Public / Private API

- Public API doesn't require authentication
    - Aka "market data", "basic api", "market api", "mapi"
    - E.g.,
        - Allow to access market data
        - Price feeds
        - Order books
        - Trade history
        - Bars

- Private API requires authentication
    - Aka "trading api", "tapi"
    - E.g.,
        - Manage personal account info
        - Query account balances
        - Trade
        - Create deposit
        - Request withdrawal
        - Query orders

- Some exchanges also expose a "merchant API" to accept crypto and fiat as payments
    - Aka "merchant", "wallet", "ecapi" (for e-commerce)

* Synch vs async calls
- `CCXT` supports asyncio
- The same methods are available but decorated with `asyncio` keyword

* Returned objects
- Public and private APIs return raw JSON objects (representing the response from
  the exchange)
- Unified APIs return a JSON object in a common format across all exchanges

In [155]:
# Print a list of all the methods in an exchange.
print(dir(ccxt.binance()))

['__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'account', 'accounts', 'addMargin', 'add_margin', 'aggregate', 'aiohttpProxy', 'aiohttpTrustEnv', 'aiohttp_proxy', 'aiohttp_trust_env', 'alias', 'amountToPrecision', 'amount_to_precision', 'api', 'apiKey', 'arrayConcat', 'array_concat', 'asyncioLoop', 'asyncio_loop', 'balance', 'base16ToBinary', 'base16_to_binary', 'base58Alphabet', 'base58Decoder', 'base58Encoder', 'base58ToBinary', 'base58_alphabet', 'base58_decoder', 'base58_encoder', 'base58_to_binary', 'base64ToBinary', 'base64ToString', 'base64_to_binary', 'base64_to_string', 'base64urlencode', 'baseCurrencies', 'base_currencies', 'binaryConcat', 'binaryConcatArray', 'binaryToBase

## Synchronous vs asynchronous

- CCXT supports async concurrency mode with async/await
- Use `asyncio` and `aiohttp`
- The methods are the same but they are decorated with `async`

```
import asyncio
import ccx.async_support as ccxt

...
```

## API parameters

- Public / private API endpoints differ from exchange to exchange
    - Most methods accept an array of key-value params
- Return a raw JSON object

- Unified API return JSON in a common format uniform across all exchanges

# Unified API

- fetch...
  - Markets
  - Currencies
  - OrderBook
  - Status
  - Trades
  - Ticker
  - Balance
- create...
  - Order
  - LimitBuyOrder / LimitSellOrder
  - MarketBuyOrder / MarketSellOrder
  - CancelOrder
- fetch orders
  - Open
  - Canceled
  - Closed
- fetch
    - my trades
    - open interest
    - transactions
    - deposit
    - withdrawals

- A `param` argument is a dictionary of exchange-specific params you want override

## Pagination
- Most exchange APIs return a certain number of the most recent objects
- You can't get all the objects in one call
  - You need to paginate, i.e., fetch portions of data one by one
  - Pagination can be performed based on id, time, or page number

# Public API

https://docs.ccxt.com/#/README?id=public-api

In [200]:
exchange.fetchMarkets()

[{'id': 'BTCUSDT',
  'lowercaseId': 'btcusdt',
  'symbol': 'BTC/USDT',
  'base': 'BTC',
  'quote': 'USDT',
  'settle': 'USDT',
  'baseId': 'BTC',
  'quoteId': 'USDT',
  'settleId': 'USDT',
  'type': 'future',
  'spot': False,
  'margin': False,
  'swap': True,
  'future': True,
  'delivery': False,
  'option': False,
  'active': True,
  'contract': True,
  'linear': True,
  'inverse': False,
  'taker': 0.0004,
  'maker': 0.0002,
  'contractSize': 1.0,
  'expiry': None,
  'expiryDatetime': None,
  'strike': None,
  'optionType': None,
  'precision': {'amount': 3, 'price': 1, 'base': 8, 'quote': 8},
  'limits': {'leverage': {'min': None, 'max': None},
   'amount': {'min': 0.001, 'max': 1000.0},
   'price': {'min': 300.5, 'max': 931777.6},
   'cost': {'min': 5.0, 'max': None},
   'market': {'min': 0.001, 'max': 1000.0}},
  'info': {'symbol': 'BTCUSDT',
   'pair': 'BTCUSDT',
   'contractType': 'PERPETUAL',
   'deliveryDate': '4133404802000',
   'onboardDate': '1569398400000',
   'status': 

In [201]:
exchange.fetchCurrencies()

In [202]:
# Not supported for Binance
# exchange.fetchStatus()

In [203]:
symbol = "BTC/USDT"
data = exchange.fetchOrderBook(symbol)
#data = exchange.fetchL2OrderBook(symbol)
print("keys=", data.keys())
for key in ["symbol", "timestamp", "datetime", "nonce", "bids"]:
    print("# %s" % key)
    hprint.pprint_color(data[key])

keys= dict_keys(['symbol', 'bids', 'asks', 'timestamp', 'datetime', 'nonce'])
# symbol
'BTC/USDT'
# timestamp
1688723745378
# datetime
'2023-07-07T09:55:45.378Z'
# nonce
32777956898
# bids
[[30281.1, 0.199],
 [30281.0, 940.294],
 [30280.9, 2.207],
 [30265.0, 106.497],
 [30263.1, 0.132],
 [30260.0, 233.5],
 [30256.7, 552.414],
 [30231.4, 0.027],
 [30222.0, 212.449],
 [30208.4, 553.298],
 [30202.1, 0.033],
 [30201.0, 0.033],
 [30200.4, 0.001],
 [30200.1, 501.32],
 [30200.0, 110.122],
 [30193.4, 0.02],
 [30190.3, 0.004],
 [30187.5, 0.004],
 [30176.5, 0.006],
 [30175.0, 1.643],
 [30174.8, 0.002],
 [30174.1, 0.002],
 [30173.7, 0.008],
 [30172.5, 0.008],
 [30169.1, 0.002],
 [30168.0, 0.002],
 [30164.0, 21.462],
 [30160.1, 20.833],
 [30160.0, 0.001],
 [30159.6, 0.002],
 [30159.5, 554.184],
 [30159.0, 1.068],
 [30153.6, 0.002],
 [30150.0, 5.054],
 [30141.6, 0.012],
 [30139.0, 0.014],
 [30137.4, 0.008],
 [30135.9, 0.004],
 [30135.8, 0.004],
 [30134.9, 0.027],
 [30134.0, 0.001],
 [30133.8, 0.006

In [204]:
data = exchange.fetchTrades(symbol)
hprint.pprint_color(data[:2])

[{'amount': 7.558,
  'cost': 228863.0422,
  'datetime': '2023-07-07T09:26:36.758Z',
  'fee': None,
  'fees': [],
  'id': '152109063',
  'info': {'T': '1688721996758',
           'a': '152109063',
           'f': '262786488',
           'l': '262786490',
           'm': True,
           'p': '30280.90',
           'q': '7.558'},
  'order': None,
  'price': 30280.9,
  'side': 'sell',
  'symbol': 'BTC/USDT',
  'takerOrMaker': 'taker',
  'timestamp': 1688721996758,
  'type': None},
 {'amount': 5.634,
  'cost': 170879.22,
  'datetime': '2023-07-07T09:26:41.558Z',
  'fee': None,
  'fees': [],
  'id': '152109064',
  'info': {'T': '1688722001558',
           'a': '152109064',
           'f': '262786491',
           'l': '262786491',
           'm': False,
           'p': '30330.00',
           'q': '5.634'},
  'order': None,
  'price': 30330.0,
  'side': 'buy',
  'symbol': 'BTC/USDT',
  'takerOrMaker': 'taker',
  'timestamp': 1688722001558,
  'type': None}]


In [205]:
symbol = "BTC/USDT"
data = exchange.fetchTicker(symbol)
hprint.pprint_color(data)

{'ask': None,
 'askVolume': None,
 'average': 30615.5,
 'baseVolume': 127267.722,
 'bid': None,
 'bidVolume': None,
 'change': -668.8,
 'close': 30281.1,
 'datetime': '2023-07-07T09:55:42.588Z',
 'high': 31190.3,
 'info': {'closeTime': '1688723742588',
          'count': '103942',
          'firstId': '262683059',
          'highPrice': '31190.30',
          'lastId': '262787014',
          'lastPrice': '30281.10',
          'lastQty': '0.001',
          'lowPrice': '29899.00',
          'openPrice': '30949.90',
          'openTime': '1688637300000',
          'priceChange': '-668.80',
          'priceChangePercent': '-2.161',
          'quoteVolume': '3872014356.85',
          'symbol': 'BTCUSDT',
          'volume': '127267.722',
          'weightedAvgPrice': '30424.17'},
 'last': 30281.1,
 'low': 29899.0,
 'open': 30949.9,
 'percentage': -2.161,
 'previousClose': None,
 'quoteVolume': 3872014356.85,
 'symbol': 'BTC/USDT',
 'timestamp': 1688723742588,
 'vwap': 30424.17}


## FetchBalance

In [206]:
symbol = "BTC/USDT"
data = exchange.fetchBalance()
hprint.pprint_color(data)

{'BNB': {'free': 8.85029226, 'total': 2.45657689, 'used': 0.0},
 'BTC': {'free': 0.06670477, 'total': 0.0, 'used': 0.0},
 'BUSD': {'free': 2069.88195453, 'total': 1000.20004, 'used': 0.0},
 'ETH': {'free': 1.08027346, 'total': 0.0, 'used': 0.0},
 'USDC': {'free': 2069.88150807, 'total': 0.0, 'used': 0.0},
 'USDT': {'free': 2111.04875759, 'total': 4586.31156979, 'used': 3987.47185241},
 'datetime': None,
 'free': {'BNB': 8.85029226,
          'BTC': 0.06670477,
          'BUSD': 2069.88195453,
          'ETH': 1.08027346,
          'USDC': 2069.88150807,
          'USDT': 2111.04875759},
 'info': {'assets': [{'asset': 'BTC',
                      'availableBalance': '0.06670477',
                      'crossUnPnl': '0.00000000',
                      'crossWalletBalance': '0.00000000',
                      'initialMargin': '0.00000000',
                      'maintMargin': '0.00000000',
                      'marginAvailable': True,
                      'marginBalance': '0.00000000',


- L1: market price only
- L2: order volume aggregated by price
- L3: each order is kept separated

## Market price

In [207]:
#symbol = exchange.symbols[0]
symbol = "BTC/USDT"
print(symbol)
orderbook = exchange.fetch_order_book(symbol)
bid = orderbook["bids"][0][0] if len(orderbook["bids"]) > 0 else None
ask = orderbook["asks"][0][0] if len(orderbook["asks"]) > 0 else None
spread = (ask - bid) if (bid and ask) else None
print(exchange.id, {"bid": bid, "ask": ask, "spread": spread})

BTC/USDT
binance {'bid': 30281.1, 'ask': 30390.0, 'spread': 108.90000000000146}


## FetchTicker()

In [208]:
symbol = "BTC/USDT"
data = exchange.fetchTicker(symbol)
hprint.pprint_color(data)

{'ask': None,
 'askVolume': None,
 'average': 30669.95,
 'baseVolume': 127277.254,
 'bid': None,
 'bidVolume': None,
 'change': -559.9,
 'close': 30390.0,
 'datetime': '2023-07-07T09:55:46.779Z',
 'high': 31190.3,
 'info': {'closeTime': '1688723746779',
          'count': '103944',
          'firstId': '262683059',
          'highPrice': '31190.30',
          'lastId': '262787016',
          'lastPrice': '30390.00',
          'lastQty': '0.001',
          'lowPrice': '29899.00',
          'openPrice': '30949.90',
          'openTime': '1688637300000',
          'priceChange': '-559.90',
          'priceChangePercent': '-1.809',
          'quoteVolume': '3872304034.33',
          'symbol': 'BTCUSDT',
          'volume': '127277.254',
          'weightedAvgPrice': '30424.16'},
 'last': 30390.0,
 'low': 29899.0,
 'open': 30949.9,
 'percentage': -1.809,
 'previousClose': None,
 'quoteVolume': 3872304034.33,
 'symbol': 'BTC/USDT',
 'timestamp': 1688723746779,
 'vwap': 30424.16}


## OHLCV bars

In [209]:
symbol = "BTC/USDT"
data = exchange.fetchOHLCV(symbol)
# O, H, L, C, V
hprint.pprint_color(data[:5])

[[1688693760000, 30040.2, 30703.2, 30015.7, 30015.7, 48.673],
 [1688693820000, 30052.4, 30703.2, 30015.7, 30703.2, 5.671],
 [1688693880000, 30703.2, 30703.2, 30328.1, 30703.2, 6.143],
 [1688693940000, 30522.5, 30703.2, 30015.7, 30074.0, 22.193],
 [1688694000000, 30069.5, 30722.5, 30015.7, 30015.7, 98.396]]


- The info from the current candle may be incomplete until the candle is closed

- Exchanges provide
    - (fast) primary data (e.g., order books, trades, fills)
        - WebSockets might be faster than REST API
    - (slow) secondary data calculated from primary data (e.g., OHLCV bars)
        - It might be faster to compute data locally

## Public trades

In [210]:
symbol = "BTC/USDT"
data = exchange.fetch_trades(symbol)
hprint.pprint_color(data[:2])

[{'amount': 0.64,
  'cost': 19379.776,
  'datetime': '2023-07-07T09:26:46.772Z',
  'fee': None,
  'fees': [],
  'id': '152109067',
  'info': {'T': '1688722006772',
           'a': '152109067',
           'f': '262786494',
           'l': '262786494',
           'm': True,
           'p': '30280.90',
           'q': '0.640'},
  'order': None,
  'price': 30280.9,
  'side': 'sell',
  'symbol': 'BTC/USDT',
  'takerOrMaker': 'taker',
  'timestamp': 1688722006772,
  'type': None},
 {'amount': 1.307,
  'cost': 39654.38,
  'datetime': '2023-07-07T09:26:52.350Z',
  'fee': None,
  'fees': [],
  'id': '152109068',
  'info': {'T': '1688722012350',
           'a': '152109068',
           'f': '262786495',
           'l': '262786495',
           'm': False,
           'p': '30340.00',
           'q': '1.307'},
  'order': None,
  'price': 30340.0,
  'side': 'buy',
  'symbol': 'BTC/USDT',
  'takerOrMaker': 'taker',
  'timestamp': 1688722012350,
  'type': None}]


## Borrow rates

- When short trading or trading with leverage on a spot market, currency must be
  borrowed

In [ ]:
# Binance doesn't support this.
#exchange.fetchBorrowRatesPerSymbol(symbol)

## Leverage tiers

In [ ]:
symbol = "BTC/USDT"
data =  exchange.fetchMarketLeverageTiers(symbol)
hprint.pprint_color(data[0])
hprint.pprint_color(data[-1])

## Funding rate

In [ ]:
data = exchange.fetchFundingRate(symbol)
hprint.pprint_color(data)

In [ ]:
data = exchange.fetchFundingRateHistory(symbol)
hprint.pprint_color(data[:3])

## Open interest

In [ ]:
# Binance doesn't support this.
# data = exchange.fetchOpenInterest(symbol)
# hprint.pprint_color(data[:3])

# Private API

https://docs.ccxt.com/#/README?id=private-api

## Authentication

In [8]:
import oms.broker.ccxt.ccxt_broker_utils as obccbrut

exchange = "binance"
contract_type = "futures"
stage = "preprod"
secret_id = 4
broker = obccbrut.get_broker(exchange, contract_type, stage, secret_id)

AttributeError: module 'im_v2.common.universe' has no attribute 'get_vendor_universe'

In [ ]:
### Accounts

* `fetchAccounts()`
- Return the accounts and sub-accounts in a JSON

### Account balance

* `fetchBalance()`
- Query for balance and get the amount of funds available

### Orders

* Query orders
- You can query orders by an id or symbol

* Place orders
- Placing orders:
  - symbol
  - side (buy or sell)
  - type: market or limit
  - amount (the base currency)

* Market orders
- Executed immediately
- You are not guaranteed that the order is executed for the price you observe
  prior to placing the order

*
- A trade is also called a "fill"
- Each trade is a result of order execution
- One order may result in several trades (i.e., an order can be filled with one
  or more trades)

### Editing orders

### My trades

* How orders are related to trades
- Each trade is a result of an order execution (matching opposing orders)
- An execution of one order can result in several trades (i.e., filled with
  multiple trades)

* Personal trades
- Typically exchanges use pagination to return all the trades

### Ledger

### Deposit

### Withdraw

### Transactions

* Trading fees
- = amount paid to the exchange
- Typically it is a percentage of volume traded

* Funding fees
- Fees for depositing and withdrawing
- Crypto transaction fees

In [ ]:
# exchange.fetchAccounts()

In [ ]:
balance = exchange.fetchBalance()

balance

In [ ]:
balance.keys()

# CCXT Pro

// https://ccxt.readthedocs.io/en/latest/ccxt.pro.manual.html

- Standard CCXT uses request-response based APIs (e.g., REST)
- CCXT Pro uses connection-based streaming API (e.g., WebSocket)

- `fetch*` methods are replaced with `watch*` methods

- CCXT Pro manages the connection-based interface transparently to the user
  - On the first call to a `watch*` method a connection is established
  - If the connection already exists, it is reused
  - The library watches the status of the connection and keeps it alive

* Sub interface
- Allows to subscribe to a stream of data and lister for it
  - E.g., streaming public market data (e.g., order book, bars)

* Pub interface
- Allows to send data requests towards the server
  - E.g., placing / cancelling orders

* Incremental data structures
- In many cases, the application listening needs to
  - keep a local snapshot of the data in memory
  - merge the updates received from the exchange server (aka "deltas")

- CCXT Pro automatically handles this